In [ ]:
from ratdata import data_manager as dm, process, ingest, plot as rdplot
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
id_ohda = 378
id_sham = 152

In [ ]:
def recording_length(recording_id):
    r = dm.RecordingFile.select().where(dm.RecordingFile.file_id == recording_id).get()
    if dm.is_recording_rejected(r.filename):
        print(f'Choose a different recording than {r.filename}')
        return None
    recording_data = ingest.read_mce_matlab_file(Path(r.dirname) / r.filename)
    if dm.is_recording_sliced(r.filename):
        recording_length = r.slice.get().length
        print(f'Selecting slice of length {recording_length} from {r.filename}')
    else:
        recording_length = recording_data.electrode_data.shape[-1] / 20000
    return recording_length

In [ ]:
min_recording_length = min([recording_length(id) for id in [id_ohda, id_sham]])

In [ ]:
fontsize_ax = 22

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.gca()
fs = 20000

colors = [
    rdplot.sham_ohda_palette['ohda'],
    rdplot.sham_ohda_palette['sham']
    ]
for i, recording_id in enumerate([id_ohda, id_sham]):
    r = dm.RecordingFile.select().where(dm.RecordingFile.file_id == recording_id).get()
    recording_data = ingest.read_mce_matlab_file(Path(r.dirname) / r.filename)
    electrode_data = recording_data.electrode_data.mean(0)
    if dm.is_recording_sliced(r.filename):
        start = int(r.slice.get().start * fs)
        end = start + int(min_recording_length * fs)
    else:
        start = 0
        end = int(min_recording_length * fs)
    data_for_psd = electrode_data[start:end]
    f, psd = signal.welch(data_for_psd, fs, nperseg=int(2*fs))
    ax.tick_params(axis='both', which='major', labelsize=fontsize_ax)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
    ax.yaxis.offsetText.set_fontsize(fontsize_ax)
    ax.plot(f[4:200], psd[4:200], color=colors[i], linewidth=4)
plt.legend(['6-OHDA', 'sham'], fontsize=fontsize_ax)

plt.savefig('plots/6ohda_vs_sham.png', bbox_inches='tight')
plt.savefig('plots/6ohda_vs_sham.svg', bbox_inches='tight')